# Data Reading

In [0]:
df=spark.read.format('parquet')\
    .option('inferSchema', True)\
        .load('abfss://bronze@adlscarsaless.dfs.core.windows.net/rawdata')

# Data Transformation

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = df.withColumn("model_category",split(col('Model_ID'),'-')[0])

In [0]:
df.withColumn("Units_Sold",col("Units_Sold").cast(StringType())).printSchema()


In [0]:
df = df.withColumn("RevPerUnit",col('Revenue')/col('Units_Sold'))
df.display()

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Day,Month,Year,BranchName,DealerName,Product_Name,model_category,RevPerUnit


# AD-HOC

In [0]:
#no. of units of each Branch got sold every year

df.groupBy("Year","BranchName").agg(sum("Units_Sold").alias("Total_Units")).sort('Year','Total_Units',ascending=[1,0])

#note: ascending=[1,0] means here:
'''
1 -> true for ascending
on column 'Year'

0 -> true for descending
on column 'Total_Units'
'''

"\n1 -> true for ascending\non column 'Year'\n\n0 -> true for descending\non column 'Total_Units'\n"

# Data Writing

In [0]:
df = df.write.format('parquet')\
    .mode('overwrite')\
        .option('path','abfss://silver@adlscarsaless.dfs.core.windows.net/carsales')\
        .save()

#note: mode('overwrite') preferred as using incremental_data also later.  s

# Querying Silver Data

In [0]:
%sql
select * from  parquet.`abfss://silver@adlscarsaless.dfs.core.windows.net/carsales`

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Day,Month,Year,BranchName,DealerName,Product_Name,model_category,RevPerUnit
